# MAST30034 Project 1
## Preprocessing Weather

In [2]:
import requests

# URLs of two datsets:
LG_2019 = "https://www.ncei.noaa.gov/data/global-hourly/access/2019/72503014732.csv"
LG_2018 = "https://www.ncei.noaa.gov/data/global-hourly/access/2018/72503014732.csv"
  
# creating HTTP response object:
r_2019 = requests.get(LG_2019) 
r_2018 = requests.get(LG_2018)

# saving the data from the URLs in dala/raw:
with open("../data/raw/LaGuardia2019.csv",'wb') as f:
    f.write(r_2019.content)

with open("../data/raw/LaGuardia2018.csv",'wb') as f:
    f.write(r_2018.content)


LaGuardia Airport was chosen as the area taken for weather measurements as out of the 3 major airports in NYC, it is the most centrally located and thus will be the best estimate of the overal weather across the city.

Creating a dataframe with all instances in one:

In [1]:
import pandas as pd
LG18 = pd.read_csv("../data/raw/LaGuardia2018.csv")
LG19 = pd.read_csv("../data/raw/LaGuardia2019.csv")
df = LG19.append(LG18)

/var/folders/ck/psrltvd10474n3pnlfszjw500000gn/T/ipykernel_48011/2719120739.py:2: DtypeWarning: Columns (44,45,58,89) have mixed types. Specify dtype option on import or set low_memory=False.
  LG18 = pd.read_csv("../data/raw/LaGuardia2018.csv")
/var/folders/ck/psrltvd10474n3pnlfszjw500000gn/T/ipykernel_48011/2719120739.py:3: DtypeWarning: Columns (44,45,46) have mixed types. Specify dtype option on import or set low_memory=False.
  LG19 = pd.read_csv("../data/raw/LaGuardia2019.csv")
/var/folders/ck/psrltvd10474n3pnlfszjw500000gn/T/ipykernel_48011/2719120739.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = LG19.append(LG18)


The datasets have approx 100 columns with highly specific weather for a variety of needs, as this project is looking at how weather and other variables effect tipping in NYC, the variables chosen were the main distinct variables commonly reported on: Wind Direction, Wind Speed, Temperature, Dew Point, Sea Level Pressure

In [2]:
df = df[['DATE', 'WND', 'TMP', 'DEW', 'SLP']]
df = df.reset_index()
total = len(df.index)
print(f"Total Instances remaining: {len(df.index)}, percentage of original: {(len(df.index)/total)*100}%")


Total Instances remaining: 28361, percentage of original: 100.0%


converting the data into a workable form

In [3]:
import numpy as np

df['date']=df['DATE'].str.extract(r'(\d{4}-\d{2}-\d{2})')

df['wind_speed']=df['WND'].str.extract(r'\d{3},\d,\w,(\d{4},\d)')
df['wind_speed'] = df['wind_speed'].str.replace(',','.',regex=True)

df['wind_direction']=df['WND'].str.extract(r'(\d{3},\d),\w,\d{4},\d')
df['wind_direction'] = df['wind_direction'].str.replace(',','.',regex=True)

df = df.rename(columns={"TMP":"temp", "DEW":"dew_point", "SLP":"pressure"})

df['temp'] = df['temp'].str.replace(',','.',regex=True)
df = df[df["temp"].str.contains("A")==False] # a few instances had random capital As in them

df['dew_point'] = df['dew_point'].str.replace(',','.',regex=True)
df['pressure'] = df['pressure'].str.replace(',','.',regex=True)

df = df.drop(['DATE', 'WND'], axis =1)
print(f"Total Instances remaining: {len(df.index)}, percentage of original: {(len(df.index)/total)*100}%")


Total Instances remaining: 28330, percentage of original: 99.89069496844259%


Converting Strings to Floats:

In [4]:
df['pressure'] = pd.to_numeric(df['pressure'], downcast='float')
df['temp'] = pd.to_numeric(df['temp'], downcast='float')
df['dew_point'] = pd.to_numeric(df['dew_point'], downcast='float')
df['wind_speed'] = pd.to_numeric(df['wind_speed'], downcast='float')
df['wind_direction'] = pd.to_numeric(df['wind_direction'], downcast='float')
df = df.drop('index', axis=1)

### Invalidating data:
#### Temp
Maximum recorded temperature in NYC was 106F and the Mminium was -15F, thus these were set as the bounds

In [5]:
df = df.drop(df[(df.temp > 106)].index)
df = df.drop(df[(df.temp < -15)].index)
print(f"Total Instances remaining: {len(df.index)}, percentage of original: {(len(df.index)/total)*100}%")

Total Instances remaining: 10186, percentage of original: 35.91551778851239%


#### Dew Point
Maximum recorded dew point in NYC was 106F and the Mminium was -49F, thus these were set as the bounds

In [6]:
df = df.drop(df[(df.dew_point > 106)].index)
df = df.drop(df[(df.dew_point < -49)].index)
print(f"Total Instances remaining: {len(df.index)}, percentage of original: {(len(df.index)/total)*100}%")

Total Instances remaining: 6786, percentage of original: 23.927224004795317%


#### Pressure
There isnt much data for the max and min sea level pressure of NYC, so the upper bounds were set to be the worlds max and min to be safe.

In [7]:
df = df.drop(df[df.pressure > 10600].index)
df = df.drop(df[df.pressure < 8700].index)
print(f"Total Instances remaining: {len(df.index)}, percentage of original: {(len(df.index)/total)*100}%")

Total Instances remaining: 5152, percentage of original: 18.165791051091286%


#### Wind Speed 
highest ever record was 185km/h during hurricane sandy  and ofcourse the slowest is set to 0.

In [8]:
df = df.drop(df[df.wind_speed > 185].index)
df = df.drop(df[df.wind_speed < 0].index)
print(f"Total Instances remaining: {len(df.index)}, percentage of original: {(len(df.index)/total)*100}%")

Total Instances remaining: 5151, percentage of original: 18.16226508233137%


#### Wind Direction 
wind direction is set in relation to compass degrees and thus the max is 360 and min is 0:

In [9]:
df = df.drop(df[df.wind_direction > 360].index)
df = df.drop(df[df.wind_direction < 0].index)
print(f"Total Instances remaining: {len(df.index)}, percentage of original: {(len(df.index)/total)*100}%")

Total Instances remaining: 4841, percentage of original: 17.069214766757167%


### Compressing Dataframe
cutting down the dataframe so that each date only appear once:

In [10]:
df = df.groupby('date').agg({'temp': 'max', # max temp was chosen as people often think about a days tempature from how hot it will get and not the average it is throughout the day
                         'dew_point':'mean', 
                         'pressure':'mean', 
                         'wind_speed': 'mean',
                         'wind_direction': 'mean'})

### Outputting Dataframe

In [11]:
df.to_parquet('../data/curated/weather', compression='gzip') 